In [1]:
# -*- coding: utf-8 -*-
import sqlite3
import os

DB_NAME = "ptt_baseball.db"

def remove_board_column():
    """
    從 articles 資料表中安全地移除 'board' 欄位。
    
    由於 SQLite 不支援直接 'DROP COLUMN'，
    標準做法是：
    1. 建立一個沒有 'board' 欄位的新資料表。
    2. 將舊表的資料複製到新表。
    3. 刪除舊表。
    4. 將新表重命名為舊表的名稱。
    """
    if not os.path.exists(DB_NAME):
        print(f"錯誤：找不到資料庫檔案 '{DB_NAME}'。")
        return

    print(f"正在處理資料庫 '{DB_NAME}'...")
    
    try:
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        
        # 使用交易確保操作的原子性，出錯時會自動還原
        cursor.execute("BEGIN TRANSACTION")
        
        print("步驟 1/4: 建立一個新的暫存資料表...")
        # 建立一個與原表結構相同但缺少 'board' 欄位的新表
        cursor.execute('''
            CREATE TABLE articles_new (
                article_id TEXT PRIMARY KEY,
                title TEXT,
                author TEXT,
                post_time TIMESTAMP,
                url TEXT UNIQUE
            )
        ''')

        print("步驟 2/4: 從舊表複製資料到新表...")
        # 將舊表的資料（除了 board 欄位）複製到新表
        cursor.execute('''
            INSERT INTO articles_new (article_id, title, author, post_time, url)
            SELECT article_id, title, author, post_time, url
            FROM articles
        ''')

        print("步驟 3/4: 刪除舊的 'articles' 資料表...")
        cursor.execute("DROP TABLE articles")

        print("步驟 4/4: 將新表重命名為 'articles'...")
        cursor.execute("ALTER TABLE articles_new RENAME TO articles")

        # 提交所有變更
        conn.commit()
        
        print("\n欄位 'board' 已成功從 'articles' 資料表中移除！")
        
        print("正在優化資料庫大小 (VACUUM)...")
        cursor.execute("VACUUM")
        conn.commit()
        print("資料庫優化完成。")

    except sqlite3.Error as e:
        print(f"\n操作失敗！發生資料庫錯誤: {e}")
        # 如果發生錯誤，conn.rollback() 會自動被呼叫
        print("所有變更已還原。")
    finally:
        if conn:
            conn.close()


if __name__ == "__main__":
    response = input(f"這個腳本將會永久從 '{DB_NAME}' 的 'articles' 資料表中刪除 'board' 欄位。\n強烈建議先備份資料庫。\n確定要繼續嗎？ (yes/no): ")
    if response.lower() == 'yes':
        remove_board_column()
    else:
        print("操作已取消。")


這個腳本將會永久從 'ptt_baseball.db' 的 'articles' 資料表中刪除 'board' 欄位。
強烈建議先備份資料庫。
確定要繼續嗎？ (yes/no):  yes


正在處理資料庫 'ptt_baseball.db'...
步驟 1/4: 建立一個新的暫存資料表...
步驟 2/4: 從舊表複製資料到新表...
步驟 3/4: 刪除舊的 'articles' 資料表...
步驟 4/4: 將新表重命名為 'articles'...

欄位 'board' 已成功從 'articles' 資料表中移除！
正在優化資料庫大小 (VACUUM)...
資料庫優化完成。


In [2]:
# -*- coding: utf-8 -*-
import sqlite3
import time
import re
from tqdm import tqdm

DB_NAME = "ptt_baseball.db"
BATCH_SIZE = 10000

def is_spam_comment(text: str) -> bool:
    """
    判斷是否為垃圾留言
    只刪除純灌水、完全沒內容的留言
    """
    if not text:
        return True
    
    text = str(text).strip()
    
    # 空白或太短（< 2 字）
    if len(text) < 2:
        return True
    
    # 純灌水詞列表（完全符合才刪除）
    spam_words = {
        '推', '噓', '→',
        'XD', 'XDDD', 'XDDDD',
        '笑死', '笑爛', '笑鼠',
        '樓上', '樓上正解', '同樓上',
        '+1', '+9', '++',
        '好喔', '好', '不錯', '讚', '讚啦', '贊',
        '哈哈', '哈哈哈', 'HAHA',
        '可憐', '可撥',
        '......', '…', '......',
        '？？？', '???',
        '！！！', '!!!',
        '先推', '推推', '必推',
        '優文', '好文',
        '錢', '發錢',
        '朝聖', '卡',
        '頭香', '搶頭香',
        '安安', '嗨',
        '通過', 'pass',
        '收到', 'ok', 'OK',
    }
    
    # 完全符合才刪除
    if text in spam_words:
        return True
    
    # <<< 新增規則：判斷是否為純網址 >>>
    if re.match(r'^(https?:\/\/|www\.)\S+$', text):
        return True
    
    # 只有標點符號
    if re.match(r'^[!?。，、；：""''…\s]+$', text):
        return True
    
    # 單一字元重複（如「哈哈哈哈哈哈哈」）
    if len(set(text)) <= 2 and len(text) >= 4:
        return True
    
    # 純數字或純英文字母（排除有意義的討論）
    if re.match(r'^[0-9]+$', text) or re.match(r'^[a-zA-Z]+$', text):
        if len(text) < 10:  # 短的純數字/字母
            return True
    
    return False


def cleanup_database():
    """清理資料庫中的垃圾留言"""
    
    print(f"開始清理資料庫：{DB_NAME}")
    print("=" * 60)
    
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    # 統計總數
    cursor.execute("SELECT COUNT(*) FROM comments")
    total_count = cursor.fetchone()[0]
    print(f"清理前總留言數：{total_count:,}")
    
    # 建立暫存表
    cursor.execute("DROP TABLE IF EXISTS comments_to_delete")
    cursor.execute("CREATE TEMP TABLE comments_to_delete (comment_id INTEGER PRIMARY KEY)")
    
    # 批次掃描
    print("\n正在掃描垃圾留言...")
    cursor.execute("SELECT comment_id, content FROM comments")
    
    delete_ids = []
    processed = 0
    
    pbar = tqdm(total=total_count, desc="掃描進度")
    
    while True:
        rows = cursor.fetchmany(BATCH_SIZE)
        if not rows:
            break
        
        for comment_id, content in rows:
            if is_spam_comment(content):
                delete_ids.append((comment_id,))
            processed += 1
        
        pbar.update(len(rows))
    
    pbar.close()
    
    # 批次寫入要刪除的 ID
    if delete_ids:
        print(f"\n準備刪除 {len(delete_ids):,} 筆垃圾留言...")
        cursor.executemany(
            "INSERT INTO comments_to_delete (comment_id) VALUES (?)",
            delete_ids
        )
        conn.commit()
    else:
        print("\n沒有需要刪除的留言")
        conn.close()
        return
    
    # 執行刪除
    print("正在刪除...")
    t0 = time.time()
    cursor.execute("""
        DELETE FROM comments 
        WHERE comment_id IN (SELECT comment_id FROM comments_to_delete)
    """)
    conn.commit()
    elapsed = time.time() - t0
    
    # 統計結果
    cursor.execute("SELECT COUNT(*) FROM comments")
    remaining_count = cursor.fetchone()[0]
    deleted_count = total_count - remaining_count
    
    # 優化資料庫
    print("\n正在優化資料庫（VACUUM）...")
    cursor.execute("VACUUM")
    
    conn.close()
    
    # 顯示結果
    print("\n" + "=" * 60)
    print("清理完成！")
    print(f"清理前：{total_count:,} 筆")
    print(f"刪除：  {deleted_count:,} 筆 ({deleted_count/total_count*100:.1f}%)")
    print(f"保留：  {remaining_count:,} 筆 ({remaining_count/total_count*100:.1f}%)")
    print(f"耗時：  {elapsed:.1f} 秒")
    print("=" * 60)


def preview_filtering(sample_size=5000):
    """預覽過濾效果"""
    print(f"預覽過濾效果（抽樣 {sample_size} 筆）...")
    print("=" * 60)
    
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    cursor.execute(f"SELECT content FROM comments ORDER BY RANDOM() LIMIT {sample_size}")
    samples = [row[0] for row in cursor.fetchall()]
    
    to_delete = [text for text in samples if is_spam_comment(text)]
    to_keep = [text for text in samples if not is_spam_comment(text)]
    
    print(f"\n抽樣結果：")
    print(f"保留：{len(to_keep)} 筆 ({len(to_keep)/sample_size*100:.1f}%)")
    print(f"刪除：{len(to_delete)} 筆 ({len(to_delete)/sample_size*100:.1f}%)")
    
    # 顯示即將刪除的樣本
    print("\n即將刪除的留言樣本（前 20 筆）：")
    for i, text in enumerate(to_delete[:20], 1):
        print(f"{i}. 「{text}」")
    
    # 顯示保留的短留言樣本（證明短留言有保留）
    print("\n保留的短留言樣本（< 10 字，前 20 筆）：")
    short_keep = [text for text in to_keep if len(str(text)) < 10][:20]
    for i, text in enumerate(short_keep, 1):
        print(f"{i}. 「{text}」")
    
    conn.close()
    print("\n" + "=" * 60)


if __name__ == "__main__":
    # 先預覽
    print("步驟 1: 預覽過濾效果\n")
    preview_filtering(sample_size=5000)
    
    # 確認
    response = input("\n確定要執行清理嗎？這會永久刪除資料。(yes/no): ")
    if response.lower() == 'yes':
        print("\n步驟 2: 執行清理\n")
        cleanup_database()
        print("\n✅ 完成！建議備份清理後的資料庫。")
    else:
        print("已取消")



步驟 1: 預覽過濾效果

預覽過濾效果（抽樣 5000 筆）...

抽樣結果：
保留：4681 筆 (93.6%)
刪除：319 筆 (6.4%)

即將刪除的留言樣本（前 20 筆）：
1. 「笑死」
2. 「推」
3. 「推」
4. 「https://i.imgur.com/vhHUyxf.jpg」
5. 「推」
6. 「茴」
7. 「順」
8. 「https://i.imgur.com/u2TEMVX.jpeg立瓜」
9. 「who」
10. 「滾」
11. 「https://youtu.be/dPGLvamtKZ0?si=bYUwZ_DZIHe4OFi」
12. 「44444」
13. 「777」
14. 「さ」
15. 「https://i.imgur.com/gXqztzD.jpeg」
16. 「猛」
17. 「講」
18. 「重」
19. 「炸」
20. 「今」

保留的短留言樣本（< 10 字，前 20 筆）：
1. 「那幹嘛用狀元選」
2. 「LUB喔」
3. 「好棒」
4. 「波士頓水有問題」
5. 「清田：又我？」
6. 「七局王」
7. 「泰山」
8. 「*0校園惡霸」
9. 「我沒有釣.jpg」
10. 「大谷！」
11. 「識相 有夠噁心」
12. 「josh粉好了啦」
13. 「大旱」
14. 「太無情了」
15. 「管這麼多」
16. 「能進就好 還挑對手」
17. 「四年會日文了吧」
18. 「不給擺爛了」
19. 「我的歐哈皮」
20. 「厲害」




確定要執行清理嗎？這會永久刪除資料。(yes/no):  yes



步驟 2: 執行清理

開始清理資料庫：ptt_baseball.db
清理前總留言數：18,529,968

正在掃描垃圾留言...



描進度: 100%|████████████████████████████████████████████████████████████████| 18529968/18529968 [00:41<00:00, 447527.09it/s]


準備刪除 1,134,097 筆垃圾留言...
正在刪除...

正在優化資料庫（VACUUM）...

清理完成！
清理前：18,529,968 筆
刪除：  1,134,097 筆 (6.1%)
保留：  17,395,871 筆 (93.9%)
耗時：  72.7 秒

✅ 完成！建議備份清理後的資料庫。
